In [104]:
import jkutils, updown_network
import os, sys, math
import numpy as np
import scipy.stats as stats
#from scipy.misc import imsave
import time
from keras import backend as K
from keras.models import load_model
import math
import shelve
import itertools
import seaborn as sns
import matplotlib as mpl

In [6]:
% matplotlib notebook
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

In [7]:
def figsize(scale,scale_height=None):
    fig_width_pt = 360.0                          # Get this from LaTeX using \the\textwidth
    inches_per_pt = 1.0/72.27                       # Convert pt to inch
    golden_mean = (np.sqrt(5.0)-1.0)/2.0            # Aesthetic ratio (you could change this)
    if scale_height == None or type(scale_height) not in (int,float):
        scale_height = golden_mean
    height_scale = scale_height
    fig_width = fig_width_pt*inches_per_pt*scale    # width in inches
    fig_height = fig_width*height_scale              # height in inches
    fig_size = [fig_width,fig_height]
    return fig_size

def savefig(filename):
    plt.savefig(os.path.join('../plots/','{}.pgf'.format(filename)))
    plt.savefig(os.path.join('../plots/','{}_pdf.pdf'.format(filename)))

In [ ]:
mpl.use('pgf')

pgf_with_latex = {                      # setup matplotlib to use latex for output
    "pgf.texsystem": "pdflatex",        # change this if using xetex or lautex
    "text.usetex": True,                # use LaTeX to write all text
    "font.family": "serif",
    "font.serif": [],                   # blank entries should cause plots to inherit fonts from the document
    "font.sans-serif": [],
    "font.monospace": [],
    "axes.labelsize": 10,               # LaTeX default is 10pt font.
    "font.size": 10,
    "legend.fontsize": 8,               # Make the legend/label fonts a little smaller
    "xtick.labelsize": 8,
    "ytick.labelsize": 8,
    "figure.figsize": figsize(0.9),     # default fig size of 0.9 textwidth
    "pgf.preamble": [
        r"\usepackage[utf8x]{inputenc}",    # use utf8 fonts becasue your computer can handle it :)
        r"\usepackage[T1]{fontenc}",        # plots will be generated using this preamble
        ]
    }
mpl.rcParams.update(pgf_with_latex)

import matplotlib.pyplot as plt

In [107]:
reload(mpl)
mpl.use('pgf')

pgf_with_latex = {                      # setup matplotlib to use latex for output
    "pgf.texsystem": "pdflatex",        # change this if using xetex or lautex
    "text.usetex": True,                # use LaTeX to write all text
    "font.family": "serif",
    "font.serif": [],                   # blank entries should cause plots to inherit fonts from the document
    "font.sans-serif": [],
    "font.monospace": [],
    "axes.labelsize": 10,               # LaTeX default is 10pt font.
    "font.size": 10,
    "legend.fontsize": 8,               # Make the legend/label fonts a little smaller
    "xtick.labelsize": 8,
    "ytick.labelsize": 8,
    "figure.figsize": figsize(0.9),     # default fig size of 0.9 textwidth
    "pgf.preamble": [
        r"\usepackage[utf8x]{inputenc}",    # use utf8 fonts becasue your computer can handle it :)
        r"\usepackage[T1]{fontenc}",        # plots will be generated using this preamble
        ]
    }
mpl.rcParams.update(pgf_with_latex)

reload(plt)

<module 'matplotlib.pyplot' from '/cvmfs/icecube.opensciencegrid.org/py2-v2/RHEL_6_x86_64/lib/python2.7/site-packages/matplotlib/pyplot.pyc'>

## load project

In [8]:
#time
today = '2017-08-24'
project_name = 'time_h012_v1'

In [108]:
file_location = '/data/user/jkager/NN_Reco/johannes/updownclassification_3/'
data_location = '/data/user/jkager/NN_Reco/training_data_20x10x60/'
test_results = 'test_results.npy'

project_folder = 'train_hist/{}/{}'.format(today, project_name)
print "looking for", project_folder
if not os.path.exists(os.path.join(file_location,project_folder)):
    print "project not found. exiting..."
    sys.exit(-1)
elif not os.path.exists(os.path.join(file_location, project_folder, test_results)):
    print "test results not found. exiting..."
    sys.exit(-1)
print "found"
shelf = shelve.open(os.path.join(file_location, project_folder, 'run_info.shlf'))
input_files = shelf['Files'].split(':')
if len(input_files) == 1: #this could be something like ['h01'] (inputformat)
    #try to decode fileinput format
    input_files = jkutils.get_filenames(input_files[0])
    for f in input_files:
        if not os.path.isfile(os.path.join(data_location, 'training_data/{}'.format(f))):
            print "file not found:", f
            print "exiting script."
            sys.exit(1)
projectpath=os.path.join(file_location,project_folder)
train_inds = shelf['Train_Inds'] 
valid_inds = shelf['Valid_Inds']
test_inds = shelf['Test_Inds']
inf_times_as = shelf['inf_times_as']
norm = shelf['time_normalized'] if shelf.has_key('time_normalized') else False
test_results = np.load(os.path.join(projectpath, test_results))
input_data, out_data, file_len = jkutils.read_files(
    input_files, 
    data_location, 
    using=shelf['using'])
res, test_out, zenith_out = test_results[0,:], test_results[1,:], test_results[2,:] #network output (0 or 1), 
                                                                                    #desired output (0 or 1),
                                                                                    #zenith (0 to pi)

looking for train_hist/2017-08-24/time_h012_v1
found


{'Files': '11069_00000-00999.h5:11069_01000-01999.h5:11069_02000-02999.h5', 'inf_times_as': 1.0, 'Valid_Inds': [(124352, 165803), (125784, 167712), (125878, 167837)], 'Test_Inds': [(165803, 207253), (167712, 209639), (167837, 209796)], 'Project': 'time_h012_v1', 'Train_Inds': [(0, 124352), (0, 125784), (0, 125878)], 'arguments': "['/mnt/lfs3/user/jkager/NN_Reco/johannes/updownclassification_3/condor_submit/../updown_network.py', '--project', 'time_h012_v1', '--continue', 'None', '--model=CNN_v1.cfg', '--input=h012', '--using=time', '--date', '2017-08-24']", 'using': 'time'}

In [54]:
model=load_model(os.path.join(projectpath, 'final_network.h5'))

In [56]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv3d_1 (Conv3D)            (None, 9, 5, 28, 64)      3136      
_________________________________________________________________
batch_normalization_1 (Batch (None, 9, 5, 28, 64)      256       
_________________________________________________________________
activation_1 (Activation)    (None, 9, 5, 28, 64)      0         
_________________________________________________________________
conv3d_2 (Conv3D)            (None, 5, 3, 14, 64)      110656    
_________________________________________________________________
batch_normalization_2 (Batch (None, 5, 3, 14, 64)      256       
_________________________________________________________________
activation_2 (Activation)    (None, 5, 3, 14, 64)      0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 13440)             0         
__________

In [63]:
ml = model.layers[0]

In [65]:
len(filter(lambda f: 'conv' in f.name, model.layers))

2

# visualize filters (see local file) #

# visualization of image with gradient ascent (see local file) #

# filter outputs for real inputs #

### choose events:

In [132]:
# choose the first n events that are correctly reconstructed
n = 50
event_mask = (np.round(res) == test_out)
c=0
for i, d in enumerate(event_mask):
    if d: 
        c+=1
    if c == n: 
        c = i
        break
event_mask[c+1:]=False
gen = updown_network.generator(n, input_data, out_data, 
                                        [list(np.arange(*test_inds[0])[event_mask])],
                                        inf_times_as=inf_times_as, normalize=norm)
gen = list(gen)
event_inputs = gen[0]
event_outputs = gen[1]
zeniths = zenith_out[event_mask]
energies = out_data[0][list(np.arange(*test_inds[0])[event_mask]),"energy"]
convlayer = filter(lambda f: 'conv' in f.name, model.layers)
filter_outputs = [{f.name : np.zeros((f.filters)) for f in convlayer} 
                  for i in range(n)]

KeyboardInterrupt: 

In [129]:
gen

<function send>

In [70]:
print len(filter_outputs), len(filter_outputs[0]), filter_outputs[0].values()[0].shape

50 2 (64, 2)


In [125]:
absolute = False

input_img=model.input
i=-1
for event in event_inputs:
    i+=1
    event = event[0]
    img_shape = event.squeeze().shape
    print img_shape
    input_img_data = event
    # we scan through every conv layer and every filter
    print('Processing event {}. Zenith: {:.2f}. Energy: {}'.format(
            i, zeniths[i], energies[i]))
    
    # we build a loss function that calculates the mean activation
    # of the nth filter of the layer considered
    for layer in convlayer:
        layer_output = layer.output
        for filter_index in range(n):
            if K.image_data_format() == 'channels_first':
                # (batch, channels, spatial_dim1, spatial_dim2, spatial_dim3)
                act = K.mean(layer_output[:, filter_index, :, :, :])
            else:
                # (batch, spatial_dim1, spatial_dim2, spatial_dim3, channels)
                act = K.mean(layer_output[:, :, :, :, filter_index])

            if absolute:
                act = K.abs(loss)

            # this function returns the act value given the input picture
            actval = K.function([input_img], [act])

            if K.image_data_format() == 'channels_first':
                input_img_data = input_img_data.reshape((1, 1)+img_shape)
            else:
                input_img_data = input_img_data.reshape((1,) + img_shape + (1,))

            val = actval([input_img_data])

            filter_outputs[i][layer.name][filter_index] = val

#np.save("kept_{}{}_{}".format(layer_name, "_abs" if absolute else "" , "-".join(project_folder.split("/")[1:])),kept_filters)

(50, 20, 10, 60)
Processing event 0. Zenith: 2.23. Energy: 800280.411572


TypeError: Bad input argument to theano function with name "/home/jkager/.local/lib/python2.7/site-packages/keras/backend/theano_backend.py:1191" at index 0 (0-based).  
Backtrace when that variable is created:

  File "/home/jkager/.local/lib/python2.7/site-packages/keras/layers/__init__.py", line 54, in deserialize
    printable_module_name='layer')
  File "/home/jkager/.local/lib/python2.7/site-packages/keras/utils/generic_utils.py", line 140, in deserialize_keras_object
    list(custom_objects.items())))
  File "/home/jkager/.local/lib/python2.7/site-packages/keras/models.py", line 1217, in from_config
    model.add(layer)
  File "/home/jkager/.local/lib/python2.7/site-packages/keras/models.py", line 439, in add
    dtype=layer.dtype, name=layer.name + '_input')
  File "/home/jkager/.local/lib/python2.7/site-packages/keras/engine/topology.py", line 1426, in Input
    input_tensor=tensor)
  File "/home/jkager/.local/lib/python2.7/site-packages/keras/legacy/interfaces.py", line 88, i

In [91]:
img_shape

(20, 10, 60)

In [97]:
event_inputs[i].reshape(1,20,10,60,1).shape

(1, 20, 10, 60, 1)